# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [3]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [4]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_TOKEN")

# Set Mapbox token
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [5]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year").drop_duplicates()

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path).drop_duplicates()

In [6]:
# Calculate average housing units per year
sfo_housing_units_per_year = sfo_data.groupby(by=[sfo_data.index])["housing_units"].mean()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [7]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_housing_units_per_year = sfo_data.groupby(by=[sfo_data.index])["housing_units"].mean().reset_index()

    # Plot the average housing units per year
    return px.bar(
        sfo_housing_units_per_year, 
        x="year",
        y="housing_units",
        title="Housing Units Per Year"
    )


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_rent_sale = sfo_data.groupby(by=[sfo_data.index])["gross_rent"].mean().reset_index()

    # Plot the average gross rent per year
    return px.bar(
        sfo_rent_sale, 
        x="year",
        y="gross_rent",
        title="Average Gross Rent in San Francisco Per Year"
    )


def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_rent_sale = sfo_data.groupby(by=[sfo_data.index])["sale_price_sqr_foot"].mean().reset_index()

    # Plot the average sales price per year
    return px.bar(
        sfo_rent_sale, 
        x="year",
        y="sale_price_sqr_foot",
        title="Average Sales Price Per Year"
    )


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    neighborhood_data = sfo_data.groupby(by=[sfo_data.index, "neighborhood"]).mean().reset_index()

    return neighborhood_data.hvplot.line(x="year", y="sale_price_sqr_foot", groupby="neighborhood", ylabel="Average Prices by Neighborhood", xlabel="Year", sort_date=True)


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # Getting the data from the top 10 expensive neighborhoods
    by_neighborhood = sfo_data.groupby(by=["neighborhood"]).mean().sort_values(by="sale_price_sqr_foot", ascending=False)
    by_neighborhood.reset_index(inplace=True)
    by_neighborhood_top10 = by_neighborhood.head(10)

    return by_neighborhood_top10.hvplot.bar(x="neighborhood", y="sale_price_sqr_foot", title="Top 10 Expensive Neighborhood in SFO", ylabel="Avg. Sale Price per Square Foot", xlabel="Neighborhood", rot=90, height=400)


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # Getting the data from the top 10 expensive neighborhoods
    by_neighborhood = sfo_data.groupby(by=["neighborhood"]).mean().sort_values(by="sale_price_sqr_foot", ascending=False)
    by_neighborhood.reset_index(inplace=True)
    by_neighborhood_top10 = by_neighborhood.head(10)

    return px.parallel_coordinates(by_neighborhood_top10, dimensions=["sale_price_sqr_foot", "housing_units", "gross_rent"], color="sale_price_sqr_foot")


def parallel_categories():
    """Parallel Categories Plot."""
    
    # Getting the data from the top 10 expensive neighborhoods
    by_neighborhood = sfo_data.groupby(by=["neighborhood"]).mean().sort_values(by="sale_price_sqr_foot", ascending=False)
    by_neighborhood.reset_index(inplace=True)
    by_neighborhood_top10 = by_neighborhood.head(10)

    return px.parallel_categories(by_neighborhood_top10, color="sale_price_sqr_foot")


def neighborhood_map():
    """Neighborhood Map"""
    
    by_neighborhood = sfo_data.groupby(by=["neighborhood"]).mean()
    by_neighborhood.reset_index(inplace=True)
    
    # Join the average values with the neighborhood locations
    mean_price_with_index = by_neighborhood.set_index("neighborhood")
    df_neighborhood_locations_with_index = df_neighborhood_locations.set_index("Neighborhood")

    mean_sales_price = (
        pd.concat([mean_price_with_index, df_neighborhood_locations_with_index], axis=1, sort=True)
        .dropna()
        .reset_index()
    )

    return px.scatter_mapbox(
        mean_sales_price,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        title="Average Sale Price Per Square Foot and Gross Rent in San Francisco",
        color_continuous_scale=px.colors.cyclical.IceFire,
        zoom=11.5,
        height=800,
        width=800,
    )

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [12]:
# Generate the plots for each tab
summary_plot = pn.Column('Data shown in the map is from 2011 to 2016. The larger the circle, the higher the average sale price per square foot. Click and drag to move around in the map. Use the scroll button to zoom in and out of the map. Hovering over the circles will display the info about the neighborhood. You will also see a set of controls on the top right if you hover over the visualization.', neighborhood_map)
parallel_plots = pn.Column('Interestingly, the neighborhood with the highest sale price per square foot has a lower gross rent than the other neighborhoods.', parallel_coordinates, parallel_categories)
neighborhood_plots = pn.Column('The top 10 most expensive neighborhoods are all averaging above a sale price of $650/square foot.', top_most_expensive_neighborhoods, average_price_by_neighborhood)
yearly_plots = pn.Column('The number of housing units per year remain relative stable however the average sale price and gross rent value has climbed steadily.', housing_units_per_year, average_sales_price, average_gross_rent)

# Generate the dashboard with tabs
dashboard = pn.Tabs(
    ('Summary', summary_plot),
    ('Yearly Analysis', yearly_plots), 
    ('Neighborhood Analysis', neighborhood_plots),
    ('Top 10 Neighborhoods', parallel_plots)
)

## Serve the Panel Dashboard

In [13]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02389')
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02540')
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02546')
        [3] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02552')
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive02413')
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive02499')
                    [0] HoloViews(DynamicMap, name='interactive02499')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
    [3] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02398')
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02404')